In [1]:
import tensorflow as tf
import keras
from keras.preprocessing import sequence
import os
import numpy as np

### Dataset

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

### Loading Your Own Data

In [3]:
import sys
sys.version

'3.7.11 (default, Jul  3 2021, 18:01:19) \n[GCC 7.5.0]'

In [4]:
## To upload own text file

#from google.colab import files
#path_to_file = list(files.upload().keys())[0] 

### Reading the contents of the file

In [5]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [6]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [7]:
vocab = sorted(set(text))

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [8]:
##lets look at how part of our text is encoded

print("Text: ", text[:13])
print("Encoded: ",text_to_int(text[:13]))

Text:  First Citizen
Encoded:  [18 47 56 57 58  1 15 47 58 47 64 43 52]


#### Reverse to Text

In [9]:
def int_to_text(ints):
  try:
    ints = int.numpy()
  except:
    pass
    return "".join(idx2char[ints])
print(int_to_text(text_as_int[:13]))

First Citizen


#### Creating Training Examples

In [10]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length + 1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

Next we can use the batch method to turn this stream of characters into batches of desired length.

In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

Now we need to use these sequences of length 101 and split them into i/p and o/p.

In [12]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text
  
dataset = sequences.map(split_input_target)

In [13]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


Finally we need to make training batches

In [14]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNITS = 1024



BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

#### Building the Model

In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([ \
                               tf.keras.layers.Embedding(vocab_size, embedding_dim, \
                                                         batch_input_shape=[batch_size, None]), \
                               tf.keras.layers.LSTM(rnn_units, \
                                                    return_sequences=True, #output at every single step
                                                    stateful = True,
                                                    recurrent_initializer = 'glorot_uniform'), \
                               tf.keras.layers.Dense(vocab_size)])
  return model

In [16]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)


In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


#### Creating a loss function

In [18]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [19]:
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-6.2382584e-03 -6.6800998e-04  7.6294597e-03 ...  1.8311475e-03
    4.8906016e-03 -1.2167565e-03]
  [-3.9360048e-03 -4.2847346e-04  4.3189330e-03 ... -1.2130500e-04
    8.1140343e-03  3.4462309e-03]
  [-6.1360812e-03 -8.8873663e-04  7.7188127e-03 ... -2.8316292e-03
    5.1555606e-03  3.8120020e-03]
  ...
  [-1.9947616e-03  9.1672624e-03  5.4239258e-03 ...  3.6907592e-03
   -7.6077720e-03  6.9335722e-03]
  [-7.6697362e-03  5.9554940e-03  1.2187602e-02 ...  4.2621018e-03
   -2.2551315e-03  4.7079227e-03]
  [-4.8351628e-03  4.2009186e-03  8.2759820e-03 ...  1.5766771e-03
    1.5062166e-03  8.3000148e-03]]

 [[-2.3097063e-03  1.5876704e-03  1.9496037e-03 ...  1.1085146e-03
   -2.7796489e-03  5.7186489e-03]
  [ 3.5501816e-03 -1.3008552e-03 -2.9526507e-03 ... -2.0848843e-03
   -1.1572511e-03  3.5406465e-03]
  [ 1.2524078e-02 -7.5015682e-03 -1.4829868e-04 ... -1.3741073e-03
   -6.0683698e-05  3.3359081e-03]
  ...
  [-5.4574180e-03  7.9689687e-03  6.7304578e-03 ... -7.2262052e

In [20]:
pred = example_batch_predictions[0]
print(len(pred))
print(pred)

100
tf.Tensor(
[[-0.00623826 -0.00066801  0.00762946 ...  0.00183115  0.0048906
  -0.00121676]
 [-0.003936   -0.00042847  0.00431893 ... -0.0001213   0.00811403
   0.00344623]
 [-0.00613608 -0.00088874  0.00771881 ... -0.00283163  0.00515556
   0.003812  ]
 ...
 [-0.00199476  0.00916726  0.00542393 ...  0.00369076 -0.00760777
   0.00693357]
 [-0.00766974  0.00595549  0.0121876  ...  0.0042621  -0.00225513
   0.00470792]
 [-0.00483516  0.00420092  0.00827598 ...  0.00157668  0.00150622
   0.00830001]], shape=(100, 65), dtype=float32)


In [21]:
time_pred = pred[0]
print(len(time_pred))
print(time_pred)

65
tf.Tensor(
[-6.2382584e-03 -6.6800998e-04  7.6294597e-03 -9.0344006e-04
  1.7512407e-03  5.7512899e-03  7.0675300e-03 -3.8826938e-03
  2.3592666e-03  3.9423229e-03 -1.5899198e-03  1.7770205e-03
  3.4033223e-03 -1.4380361e-03 -8.4194454e-04  3.0876156e-03
 -1.7554397e-03 -3.1225372e-03  1.1666000e-03 -6.0386835e-03
  1.3185857e-03  2.6633663e-03  4.7901939e-03  3.1272133e-03
  3.4269546e-03  1.2994278e-06 -8.8468194e-04 -4.7616879e-03
  2.2039728e-04  1.5089363e-03 -1.5572517e-03  1.8639646e-03
 -1.7093993e-03 -1.9745931e-03  2.1474373e-03 -2.0857202e-05
 -2.3082811e-03 -5.1720436e-03 -1.8614073e-03 -7.9326564e-03
 -1.1212047e-03  4.2466954e-03  1.0240322e-03 -1.5289180e-03
  6.7181559e-03  1.4912826e-04 -1.6386184e-03 -7.5525278e-04
  5.3186938e-03  1.6566128e-03 -2.5355041e-03 -4.7601825e-03
 -2.5302004e-03 -1.2113551e-03  5.8800508e-03 -3.1809940e-03
 -4.5720870e-03 -3.7516796e-03  2.3294070e-03  1.7864101e-03
 -5.5894400e-03  1.9605085e-03  1.8311475e-03  4.8906016e-03
 -1.216756

In [22]:
sample_indices = tf.random.categorical(pred, num_samples = 1)

sample_indices = np.reshape(sample_indices, (1, -1))[0]
predicted_chars = int_to_text(sample_indices)

predicted_chars

"U'TEjoLG- B'iQhQCXbjfZpOJx3 b.&LfYgnteOg;Pix. -zca?SekYK3:XQw dqI,!keEVcHpuesHETanxhLZTe;Tz;L!E3rBQg"

In [23]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

#### Compiling the Model

In [24]:
model.compile(optimizer = 'adam', loss=loss)

#### Creating checkpoints

In [25]:
checkpoint_dir = "./training_checkpoints"

checkpoint_prefix = os.path.join(checkpoint_dir,"ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint( \
                                                         filepath = checkpoint_prefix, \
                                                         save_weights_only = True)

#### Training

In [26]:
history = model.fit(data, epochs=20, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 15s 71ms/step - loss: 2.5930
Epoch 2/20
172/172 [==============================] - 14s 74ms/step - loss: 1.8862
Epoch 3/20
172/172 [==============================] - 14s 77ms/step - loss: 1.6378
Epoch 4/20
172/172 [==============================] - 14s 74ms/step - loss: 1.5065
Epoch 5/20
172/172 [==============================] - 14s 74ms/step - loss: 1.4257
Epoch 6/20
172/172 [==============================] - 13s 72ms/step - loss: 1.3690
Epoch 7/20
172/172 [==============================] - 14s 74ms/step - loss: 1.3242
Epoch 8/20
172/172 [==============================] - 14s 74ms/step - loss: 1.2841
Epoch 9/20
172/172 [==============================] - 14s 75ms/step - loss: 1.2478
Epoch 10/20
172/172 [==============================] - 14s 74ms/step - loss: 1.2132
Epoch 11/20
172/172 [==============================] - 14s 73ms/step - loss: 1.1775
Epoch 12/20
172/172 [==============================] - 14s 74ms/step - loss: 1.1399
E

#### Loading the Model

In [27]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [28]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [29]:
### Need to fix below code

#checkpoint_num = 2
#model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
#model.build(tf.TensorShape([1, None]))

#### Generating text

In [30]:
def generate_text(model, start_string):
  num_generate = 800

  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  temperature = 1.0

  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)

    predictions = tf.squeeze(predictions, 0)

    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples = 1)[-1, 0].numpy()

    input_eval = tf.expand_dims([predicted_id], 0 )

    text_generated.append(idx2char[predicted_id])
    
  return (start_string + ''.join(text_generated))

In [32]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: romeo
romeout in the middle of my mouth
a-morrow with occasaff.
I am sorry that then we shall perform
To play them at, 'em, as thou know'st he knew,
Or you, pronounced that bar framed, was Catesby doe:
Fin, sir, fooly as the father Edward's son,
Thy father, beaking not of my faithful; nor then but Friar,' dear spider,
R than my love I well depend me.

PETRUCHIO:
Why, say you may do so, and that I may say
something and my tradition maids before him,
Though on human hath made fift
Aga'nt me as he then mine, and put on nothing build one that made
curded point that fear'd of hatten heard their suns;
Forswear Bohemia, came to advise my deliverath
dare narrow by nine in Bianco's coward?

PRINCE:
Partly for the musar:
Mark him for your cause, yet we resist it borne to make a woman.

ESCALUS:
My lord, I hear
